In [62]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, scale, PolynomialFeatures, OneHotEncoder, OrdinalEncoder

from IPython.display import HTML, display_html, display

from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier

In [87]:
# ['sample_submission.csv', 'greeks.csv', 'train.csv', 'test.csv']

X_train = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv", sep=",")
X_greeks = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/greeks.csv", sep=",")
sample_submission = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv", sep=",")

print(X_train.shape)
print(X_train.columns)

X_train.describe()

(617, 58)
Index(['Id', 'AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN',
       'BP', 'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS',
       'CU', 'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
       'EB', 'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI',
       'FL', 'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL', 'Class'],
      dtype='object')


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
count,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,...,616.000000,617.000000,615.000000,617.000000,617.000000,617.000000,617.000000,617.000000,616.000000,617.000000
mean,0.477149,3502.013221,118.624513,38.968552,10.128242,5.545576,0.060320,10.566447,8.053012,5350.388655,...,5.433199,3.533905,0.421501,20.724856,131.714987,14679.595398,31.489716,50.584437,8.530961,0.175041
std,0.468388,2300.322717,127.838950,69.728226,10.518877,2.551696,0.416817,4.350645,65.166943,3021.326641,...,11.496257,50.181948,1.305365,9.991907,144.181524,19352.959387,9.864239,36.266251,10.327010,0.380310
min,0.081187,192.593280,85.200147,3.177522,8.138688,0.699861,0.025578,3.396778,1.229900,1693.624320,...,0.173229,0.497060,0.067730,4.102182,72.611063,13.038894,9.432735,0.897628,0.001129,0.000000
25%,0.252107,2197.345480,85.200147,12.270314,8.138688,4.128294,0.025578,8.129580,1.229900,4155.702870,...,0.173229,0.497060,0.067730,14.036718,72.611063,2798.992584,25.034888,23.011684,0.124392,0.000000
50%,0.354659,3120.318960,85.200147,20.533110,8.138688,5.031912,0.025578,10.461320,1.229900,4997.960730,...,3.028141,1.131000,0.250601,18.771436,72.611063,7838.273610,30.608946,41.007968,0.337827,0.000000
75%,0.559763,4361.637390,113.739540,39.139886,8.138688,6.431634,0.036845,12.969516,5.081244,6035.885700,...,6.238814,1.512060,0.535067,25.608406,127.591671,19035.709240,36.863947,67.931664,21.978000,0.000000
max,6.161666,28688.187660,1910.123198,630.518230,178.943634,38.270880,10.315851,38.971568,1463.693448,53060.599240,...,137.932739,1244.227020,31.365763,135.781294,1497.351958,143790.071200,81.210825,191.194764,21.978000,1.000000


In [43]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 58 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      617 non-null    object 
 1   AB      617 non-null    float64
 2   AF      617 non-null    float64
 3   AH      617 non-null    float64
 4   AM      617 non-null    float64
 5   AR      617 non-null    float64
 6   AX      617 non-null    float64
 7   AY      617 non-null    float64
 8   AZ      617 non-null    float64
 9   BC      617 non-null    float64
 10  BD      617 non-null    float64
 11  BN      617 non-null    float64
 12  BP      617 non-null    float64
 13  BQ      557 non-null    float64
 14  BR      617 non-null    float64
 15  BZ      617 non-null    float64
 16  CB      615 non-null    float64
 17  CC      614 non-null    float64
 18  CD      617 non-null    float64
 19  CF      617 non-null    float64
 20  CH      617 non-null    float64
 21  CL      617 non-null    float64
 22  CR

In [80]:
num_C = 0
for i in  X_train.columns:
    print (i+"("+str(num_C)+")")
    print (X[i].value_counts())
    print ("\n")
    num_C=num_C+1

Id(0)
000ff2bfdfe9    1
a20c68851e9b    1
9ee887ad93d5    1
9ef1886b810c    1
9f10f72ca154    1
               ..
50183292c0e7    1
508bb5c5cf00    1
51111505c6f2    1
514d2ac4fdb5    1
ffcca4ded3bb    1
Name: Id, Length: 617, dtype: int64


AB(1)
0.260653    11
0.282018    10
0.337567     9
0.324748     9
0.235015     9
            ..
0.224332     1
0.952879     1
1.066113     1
0.525579     1
0.435846     1
Name: AB, Length: 217, dtype: int64


AF(2)
192.59328     19
3109.03329     1
1076.39180     1
4612.74264     1
7057.21852     1
              ..
2056.42404     1
6162.10892     1
4182.68565     1
1642.84362     1
2672.53426     1
Name: AF, Length: 599, dtype: int64


AH(3)
85.200147     388
128.318406      2
91.801569       2
123.807144      2
226.129185      1
             ... 
94.666830       1
94.335888       1
132.881922      1
132.350673      1
546.663930      1
Name: AH, Length: 227, dtype: int64


AM(4)
630.518230    5
13.217996     2
35.110689     2
14.221424     2
8.3371

In [81]:
# Index(['Id', 'AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 
#        'BC', 'BD ', 'BN','BP', 'BQ', 'BR', 'BZ', 
#        'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS','CU', 'CW ', 
#        'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
#        'EB', 'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 
#        'FC', 'FD ', 'FE', 'FI', 'FL', 'FR', 'FS', 
#        'GB', 'GE',  'GF',  'GH',  'GI',  'GL', 
#        'Class'], 




transformersX = [
  ("Id",                                              "drop",                                               [0]),
    
  ("AB",                                              StandardScaler(),                                     [1]),
  ("AF",                                              StandardScaler(),                                     [2]),
  ("AH",                                              StandardScaler(),                                     [3]),
  ("AM",                                              StandardScaler(),                                     [4]),
  ("AR",                                              StandardScaler(),                                     [5]),
  ("AX",                                              StandardScaler(),                                     [6]), 
  ("AY",                                              StandardScaler(),                                     [7]),
  ("AZ",                                              StandardScaler(),                                     [8]),
    
  ("BC",                                              StandardScaler(),                                     [9]),
  ("BD",                                              StandardScaler(),                                     [10]),
  ("BN",                                              StandardScaler(),                                     [11]),
  ("BP",                                              StandardScaler(),                                     [12]),
  ("BQ",                                              StandardScaler(),                                     [13]),
  ("BR",                                              StandardScaler(),                                     [14]),
  ("BZ",                                              StandardScaler(),                                     [15]),
    
  ("CB",                                              StandardScaler(),                                     [16]),
  ("CC",                                              StandardScaler(),                                     [17]),
  ("CD",                                              StandardScaler(),                                     [18]),
  ("CF",                                              StandardScaler(),                                     [19]),
  ("CH",                                              StandardScaler(),                                     [20]),
  ("CL",                                              StandardScaler(),                                     [21]),
  ("CR",                                              StandardScaler(),                                     [22]),
  ("CS",                                              StandardScaler(),                                     [23]),
  ("CU",                                              StandardScaler(),                                     [24]),
  ("CW",                                              StandardScaler(),                                     [25]),
    
  ("DA",                                              StandardScaler(),                                     [26]),
  ("DE",                                              StandardScaler(),                                     [27]),
  ("DF",                                              StandardScaler(),                                     [28]),
  ("DH",                                              StandardScaler(),                                     [29]),
  ("DI",                                              StandardScaler(),                                     [30]),
  ("DL",                                              StandardScaler(),                                     [31]),
  ("DN",                                              StandardScaler(),                                     [32]),
  ("DU",                                              StandardScaler(),                                     [33]),
  ("DV",                                              StandardScaler(),                                     [34]),
  ("DY",                                              StandardScaler(),                                     [35]),

  ("EB",                                              StandardScaler(),                                     [36]),
  ("EE",                                              StandardScaler(),                                     [37]),
  ("EG",                                              StandardScaler(),                                     [38]),
  ("EH",                                              StandardScaler(),                                     [39]),
  ("EJ",                                              "drop",                                               [40]),
  ("EL",                                              StandardScaler(),                                     [41]),
  ("EP",                                              StandardScaler(),                                     [42]),
  ("EU",                                              StandardScaler(),                                     [43]),
  
  ("FC",                                              StandardScaler(),                                     [44]),
  ("FD",                                              StandardScaler(),                                     [45]),
  ("FE",                                              StandardScaler(),                                     [46]),    
  ("FI",                                              StandardScaler(),                                     [47]),
  ("FL",                                              StandardScaler(),                                     [48]),
  ("FR",                                              StandardScaler(),                                     [49]),
  ("FS",                                              StandardScaler(),                                     [50]),
  
  ("GB",                                              StandardScaler(),                                     [51]),
  ("GE",                                              StandardScaler(),                                     [52]),
  ("GF",                                              StandardScaler(),                                     [53]),
  ("GH",                                              StandardScaler(),                                     [54]),
  ("GI",                                              StandardScaler(),                                     [55]),
  ("GL",                                              StandardScaler(),                                     [56]),
  
  ("Class",                                           "passthrough",                                        [57]),
]

X_T = ColumnTransformer(transformers=transformersX).fit_transform(X_train)

In [82]:
X["Class"].value_counts()

# Oversampling?

0    509
1    108
Name: Class, dtype: int64

In [85]:
_X_T = X_T

_Score = X["Class"]

clf_MLP = MLPClassifier(random_state=1, max_iter=250)

display(HTML('<h3>5. MLPClassifier (Neural-Network)</h3>'))

clf_MLP.fit(_X_T, _Score)
pred_MLP = clf_MLP.predict(Y_T)

solucion_MLP = pd.DataFrame(data = { "ID" : Y["ID"], "Class": pred_MLP })
solucion_MLP.to_csv("submission.csv", index = False)

display(solucion_MLP)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices